In [1]:
# set current working directory
import os
if os.name == 'posix':
    %cd ~/workspace/python/CNStocks
else:
    %cd D:\workspace\python\CNStocks
        
import h5py

from types import SimpleNamespace
%load_ext Cython

D:\workspace\python\CNStocks


In [2]:
from libs.cython.compute import compute_stats

In [3]:
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

In [4]:
msl = SimpleNamespace()
fsl = SimpleNamespace()
stat = SimpleNamespace()
basics = SimpleNamespace()
file = os.path.join(os.getcwd(), 'hdf5', '20190812.hdf5')

with h5py.File(file, "a") as f:
    codes = np.copy(np.char.decode(f[u'codes'], 'utf-8'))
#     setup_arrays(codes)

    names = np.copy(np.char.decode(f[u'names'], 'utf-8'))
    msl.data = np.copy(f[u'msl'][:,:,:])
    fsl.data = np.copy(f[u'fsl'][:,:,:])
    stat.data = np.copy(f[u'stat'][:,:,:])
    basics.data = np.copy(f[u'basics'][:,:])

In [6]:
index_ms = 2999
index_fs = 249

In [9]:
ms = msl.data[index_ms, :, :]
ms

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [12]:
fs = fsl.data[index_fs, :, :]

In [13]:
b = basics.data[:, :]

In [14]:
st = stat.data[index_fs, :, :]

In [33]:
ms1p = msl.data[index_ms-12, :, :]

In [34]:
index_fs5p = min(index_fs - 5, 9)
fs5p = fsl.data[index_fs5p, :, :]
fs5p[:,0] = fs[:,0]*0.95

In [35]:
%%cython
import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport isnan

ctypedef np.float32_t DTYPE_t

@cython.boundscheck(False)
@cython.cdivision
@cython.wraparound(False)
cpdef compute_stats(DTYPE_t[:,:] ms, DTYPE_t[:,:] fs, DTYPE_t[:,:] b, DTYPE_t[:,:] st, DTYPE_t[:,:] ms1p, DTYPE_t[:,:] fs5p, Py_ssize_t index_fs):
    
    # ms
    cdef Py_ssize_t idx_open = 0
    cdef Py_ssize_t idx_now = 2
    cdef Py_ssize_t idx_close = 1
    cdef Py_ssize_t idx_turnover = 5
    cdef Py_ssize_t idx_volume = 6
    
    cdef Py_ssize_t idx_zf = 0
    cdef Py_ssize_t idx_jj = 1
    cdef Py_ssize_t idx_lb = 2
    cdef Py_ssize_t idx_zs = 3
    cdef Py_ssize_t idx_zt = 4
    cdef Py_ssize_t idx_fsto = 5
    
    cdef Py_ssize_t rows = ms.shape[0]
    cdef Py_ssize_t cols = ms.shape[1]
    
    cdef Py_ssize_t interval = min(index_fs - 9, 1)
    
    cdef list indices_zt = []
    for i in range(rows):
        
        fs[i,0] = ms[i,2]
        if index_fs == 0:
            fs[i,1] = ms[i,5]
        else:
            fs[i,1] = ms[i,5] - ms1p[i,5]
        
        if ms[i,idx_open]:
            st[i,idx_zf] = 100*(ms[i,idx_now]/ms[i,idx_close]-1)

        if ms[i,idx_turnover]:
            st[i,idx_jj] = ms[i,idx_volume]/ms[i,idx_turnover]
            st[i,idx_zs] = 100*(ms[i,idx_now]/fs5p[i, 0]-1)
            if ms[i,idx_now] - b[i,0] > -0.0001:
                indices_zt.append(i)

        if b[i,2] and ms[i,idx_turnover]:
            st[i,idx_lb] = ms[i,idx_turnover]/interval/b[i,2]

    return indices_zt

In [41]:
%%timeit -n1000
compute_stats(ms, fs, b, st, ms1p, fs5p, index_fs)

52.6 µs ± 2.34 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
import numpy as np

In [ ]:
st.fill(np.nan)
st

In [ ]:
st

In [15]:
ms

array([[13.90, 13.54, 14.38, ..., 13.85, 233071504.00, 3313302016.00],
       [27.53, 27.36, 27.65, ..., 27.36, 27846622.00, 767941248.00],
       [18.39, 18.16, 18.13, ..., 18.03, 355936.00, 6478991.00],
       ...,
       [217.00, 68.58, 243.88, ..., 215.65, 16602102.00, 4071389440.00],
       [97.00, 100.55, 90.20, ..., 90.10, 6603556.00, 619324032.00],
       [72.49, 72.50, 67.99, ..., 67.55, 12577047.00, 880639808.00]],
      dtype=float32)

In [ ]:
names

In [16]:
ms

array([[13.90, 13.54, 14.38, ..., 13.85, 233071504.00, 3313302016.00],
       [27.53, 27.36, 27.65, ..., 27.36, 27846622.00, 767941248.00],
       [18.39, 18.16, 18.13, ..., 18.03, 355936.00, 6478991.00],
       ...,
       [217.00, 68.58, 243.88, ..., 215.65, 16602102.00, 4071389440.00],
       [97.00, 100.55, 90.20, ..., 90.10, 6603556.00, 619324032.00],
       [72.49, 72.50, 67.99, ..., 67.55, 12577047.00, 880639808.00]],
      dtype=float32)

In [31]:
'600610' in codes

False

In [ ]:
np.get_include()